In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import missingno
import time
from tqdm import tqdm,tqdm_notebook,tqdm_pandas
%matplotlib inline

## 读取静态文件

In [16]:
ad_static_fea  = pd.read_csv('./dataset/ad_static_feature.out',
                             sep='\t',
                             header=None,
                             names=['广告id', '创建时间','广告账户id','商品id','商品类型','广告行业id','素材尺寸'])

/mnt/7D8C5D9D3CC9F65C/lph/torch_py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
ad_static_fea['创建时间'] = ad_static_fea['创建时间'].apply(lambda x: time.strftime("%Y-%m-%d",time.localtime(x))).values

In [18]:
ad_static_fea.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,106452,2018-06-26,22226,16088,13,225,NaN
1,233649,2018-09-29,25681,7356,13,136,1
2,547531,2019-02-21,20696,-1,1,186,40
3,707841,2019-03-06,3968,-1,3,186,40
4,457009,2019-02-18,23614,7447,13,172,NaN


## 对曝光广告文件进行挖掘

In [20]:
temp_totalExposureLog_pro =  pd.read_csv('./dataset/totalExposureLog_pro.csv',sep=',')[['曝光广告id','曝光广告出价bid', '曝光广告素材尺寸','pctr','equality_ecpm']]

In [21]:
temp_totalExposureLog_pro.head()

,曝光广告id,曝光广告出价bid,曝光广告素材尺寸,pctr,equality_ecpm
0,451525,46,50,47.217,944.34
1,451525,56,50,39.129,782.58
2,451525,18,50,119.905,2398.10
3,451525,100,50,21.763,435.26
4,451525,20,50,107.212,2144.24


In [25]:
temp_totalExposureLog_math_fea = temp_totalExposureLog_pro.groupby(['曝光广告id']).agg({'曝光广告出价bid':[min, max, np.mean,np.std],
                                                       '曝光广告素材尺寸':[max, np.mean],
                                                       'pctr':[max, min, np.mean, np.std],
                                                       })

In [57]:
bid_fea = temp_totalExposureLog_math_fea['曝光广告出价bid'].reset_index()
bid_fea.columns = ['曝光广告id', 'bid_min', 'bid_max', 'bid_mean', 'bid_std']

In [63]:
ad_size_fea = temp_totalExposureLog_math_fea['曝光广告素材尺寸'].reset_index()
ad_size_fea.columns = ['曝光广告id', 'ad_size_max', 'ad_size_mean']

In [65]:
pctr_fea = temp_totalExposureLog_math_fea['pctr'].reset_index()
pctr_fea.columns = ['曝光广告id',  'pctr_max','pctr_min', 'pctr_mean', 'pctr_std']

In [66]:
bid_fea.head()

,曝光广告id,bid_min,bid_max,bid_mean,bid_std
0,1,24,1594,306.629630,420.721098
1,2,68,2969,440.037037,541.292953
2,3,10,68,23.166667,18.393840
3,4,10,1403,173.282392,132.116900
4,5,12,12,12.000000,NaN


In [67]:
ad_size_fea.head()

,曝光广告id,ad_size_max,ad_size_mean
0,1,64,64.0
1,2,36,36.0
2,3,64,64.0
3,4,64,64.0
4,5,50,50.0


In [69]:
pctr_fea.head()

,曝光广告id,pctr_max,pctr_min,pctr_mean,pctr_std
0,1,6.960,1.225,3.111519,1.255610
1,2,69.826,1.103,7.171383,7.878272
2,3,89.268,2.725,16.047167,25.080070
3,4,62.071,0.010,7.591223,5.730947
4,5,268.088,268.088,268.088000,NaN


In [70]:
fea_select_1 = pd.merge(bid_fea, ad_size_fea, on='曝光广告id')
fea_select_1 = pd.merge(fea_select_1, pctr_fea, on='曝光广告id')

In [77]:
fea_select_1  =fea_select_1.rename(index=str, columns={"曝光广告id": "广告id"})

In [9]:
## 广告id 创建时间 素材尺寸 广告行业id 商品类型 商品id 广告账户id 投放时段 人群定向 出价

In [79]:
ad_fea = pd.merge(ad_static_fea,fea_select_1, on='广告id',how='outer')

In [80]:
ad_fea

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,bid_min,bid_max,bid_mean,bid_std,ad_size_max,ad_size_mean,pctr_max,pctr_min,pctr_mean,pctr_std
0,106452,2018-06-26,22226.0,16088,13.0,225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,233649,2018-09-29,25681.0,7356,13.0,136,1,100.0,100.0,100.000000,NaN,1.0,1.0,0.841,0.841,0.841000,NaN
2,547531,2019-02-21,20696.0,-1,1.0,186,40,100.0,512.0,485.651163,90.166746,40.0,40.0,28.486,1.483,7.374188,5.335201
3,707841,2019-03-06,3968.0,-1,3.0,186,40,74.0,425.0,205.875000,142.742963,40.0,40.0,12.008,0.905,5.179125,3.513139
4,457009,2019-02-18,23614.0,7447,13.0,172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,733436,2019-03-19,22405.0,31722,5.0,117,64,15.0,794.0,118.551282,139.377619,64.0,64.0,63.112,0.076,6.941026,10.000894
6,249105,2019-03-15,11360.0,29999,18.0,145,44,51.0,109.0,72.666667,22.277043,44.0,44.0,33.469,14.670,20.021167,6.900286
7,160014,2019-03-14,6441.0,2373,18.0,198,36,13.0,282.0,71.915254,45.216067,36.0,36.0,29.112,4.463,15.680068,5.886980
8,541096,2019-03-13,5117.0,220,5.0,232,44,8.0,174.0,96.416667,42.904669,44.0,44.0,71.036,6.627,19.813083,19.477155
9,634000,2019-03-11,28588.0,20164,18.0,76,64,10.0,40.0,17.000000,8.514693,64.0,64.0,8.430,1.568,3.624941,1.624254


In [81]:
ad_static_fea.shape

(735911, 7)

In [82]:
fea_select_1.shape

(509280, 11)

In [84]:
len(np.unique(temp_totalExposureLog_pro['曝光广告id']))

509280

In [85]:
ad_fea.to_csv('./ad_fea_li.csv',index=None)